<a href="https://colab.research.google.com/github/Bouza1/ImageZooming/blob/master/builds_format.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BU Buildings

In [102]:
import pandas as pd
import numpy as np
import json

### Utilitiy

In [103]:
energy = {
    'electricity':{'price':0.45, 'unit':'kwh'},
    'gas':{'price':0.107, 'unit':'kwh'},
    'water':{'price':0.370, 'unit':'m3'}
}

carbon_cost = {
    'electricity':{'carbon':0.193, 'unit':'kgCO2e / kWh'},
    'gas':{'carbon':0.180, 'unit':'kgCO2e / kWh'},
    'water':{'carbon':0.149, 'unit':'kgCO2e / kWh'}
}

In [104]:
buildings = {
    'PGB':{
        'dataset':'PGB.csv',
        'mains_electric':True,
        'solar_electric':True,
        'mains_gas':True,
        'heat_pump':True,
        'mains_water': False,
        'rain_harvest':True,
        'consumption_totals':{
            'electric_consumption':0,
            'heat_consumption':0,
            'water_consumption':0
        },
        'footprint':{
            'electric_co2':0,
            'heating_co2':0,
            'water_co2':0
        },
        'cost_totals':{
            'electric_cost':0,
            'heating_cost':0,
            'water_cost':0
        }
    },
    'DH':{
        'dataset':'DH.csv',
        'mains_electric':True,
        'solar_electric':True,
        'mains_gas':True,
        'heat_pump':True,
        'mains_water': False,
        'rain_harvest':True,
        'consumption_totals':{
            'electric_consumption':0,
            'heat_consumption':0,
            'water_consumption':0
        },
        'footprint':{
            'electric_co2':0,
            'heating_co2':0,
            'water_co2':0
        },
        'cost_totals':{
            'electric_cost':0,
            'heating_cost':0,
            'water_cost':0
        }
    },
    'KH':{
        'dataset':'KH.csv',
        'mains_electric':True,
        'solar_electric':True,
        'mains_gas':True,
        'heat_pump':True,
        'mains_water': False,
        'rain_harvest':True,
        'consumption_totals':{
            'electric_consumption':0,
            'heat_consumption':0,
            'water_consumption':0
        },
        'footprint':{
            'electric_co2':0,
            'heating_co2':0,
            'water_co2':0
        },
        'cost_totals':{
            'electric_cost':0,
            'heating_cost':0,
            'water_cost':0
        }
    },
    'FB':{
        'dataset':'FB.csv',
        'mains_electric':True,
        'solar_electric':True,
        'mains_gas':True,
        'heat_pump':True,
        'mains_water': False,
        'rain_harvest':True,
        'consumption_totals':{
            'electric_consumption':0,
            'heat_consumption':0,
            'water_consumption':0
        },
        'footprint':{
            'electric_co2':0,
            'heating_co2':0,
            'water_co2':0
        },
        'cost_totals':{
            'electric_cost':0,
            'heating_cost':0,
            'water_cost':0
        }
    }
}

In [105]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [106]:
def load_dataset(name):
  return pd.read_csv(f'/content/drive/MyDrive/{name}')

In [107]:
def format_dataset(dataset):
  ## Change Column Names
  if 'Incoming Grid electricity (kWh)' in dataset.columns:
    dataset.rename(columns={'Incoming Grid electricity (kWh)': 'Mains Electric'}, inplace=True)
  if 'Solar Electricity Production (kWh)' in dataset.columns:
    dataset.rename(columns={'Solar Electricity Production (kWh)': 'Solar Electric'}, inplace=True)
  if 'Incoming mains Gas (kWh)' in dataset.columns:
    dataset.rename(columns={'Incoming mains Gas (kWh)': 'Mains Gas'}, inplace=True)
  if 'Incoming Mains Water  (m3)' in dataset.columns:
    dataset.rename(columns={'Incoming Mains Water  (m3)': 'Mains Water'}, inplace=True)
  if 'Ground Source Heat Pump heating energy (kWh)' in dataset.columns:
    dataset.rename(columns={'Ground Source Heat Pump heating energy (kWh)': 'Heat Pump'}, inplace=True)
  if 'Rainwater harvesting (m3)' in dataset.columns:
    dataset.rename(columns={'Rainwater harvesting (m3)': 'Rain Harvest'}, inplace=True)
  ## Replace Blank Values With 0
  dataset.replace('', np.nan, inplace=True)
  dataset.fillna(0, inplace=True)

  return dataset

In [108]:
for key, building in buildings.items():
    building['dataset'] = format_dataset(load_dataset(building['dataset']))

In [109]:
# buildings['PGB']['dataset'].info()
# buildings['DH']['dataset'].info()
# buildings['KH']['dataset'].info()
# buildings['FB']['dataset'].info()

In [110]:
def make_energy_consumptions(building):
  temp_df = building['dataset']
  # Electric Consumption
  if 'Mains Electric' in temp_df.columns:
     building['consumption_totals']['electric_consumption']= round((building['consumption_totals']['electric_consumption'] + temp_df['Mains Electric'].sum()), 2)
  if 'Solar Electric' in temp_df.columns:
    building['consumption_totals']['electric_consumption'] = round((building['consumption_totals']['electric_consumption'] + temp_df['Solar Electric'].sum()),2)
  # Heating Consumption
  if 'Mains Gas' in temp_df.columns:
    building['consumption_totals']['heat_consumption'] = round(building['consumption_totals']['heat_consumption'] + temp_df['Mains Gas'].sum(),2)
  if 'Heat Pump'in temp_df.columns:
    building['consumption_totals']['heat_consumption'] = round((building['consumption_totals']['heat_consumption'] + temp_df['Heat Pump'].sum()),2)
  # Water Consumption
  if 'Mains Water' in temp_df.columns:
    building['consumption_totals']['water_consumption'] = round((building['consumption_totals']['water_consumption'] + temp_df['Mains Water'].sum()),2)
  if 'Rain Harvest'in temp_df.columns:
    building['consumption_totals']['water_consumption'] = round((building['consumption_totals']['water_consumption'] + temp_df['Rain Harvest'].sum()),2)



In [111]:
for key, building in buildings.items():
  make_energy_consumptions(building)
  print(f"{key} - Consumption Totals")
  print(f"Total Electric Consumption: {building['consumption_totals']['electric_consumption']}")
  print(f"Total Heating Consumption: {building['consumption_totals']['heat_consumption']}")
  print(f"Total Water Consumption: {building['consumption_totals']['water_consumption']}")

PGB - Consumption Totals
Total Electric Consumption: 707810.28
Total Heating Consumption: 535457.36
Total Water Consumption: 0
DH - Consumption Totals
Total Electric Consumption: 266511.0
Total Heating Consumption: 342284.42
Total Water Consumption: 1201.6
KH - Consumption Totals
Total Electric Consumption: 126365.5
Total Heating Consumption: 21420.3
Total Water Consumption: 203.49
FB - Consumption Totals
Total Electric Consumption: 582220.43
Total Heating Consumption: 151366.34
Total Water Consumption: 252576.0


In [112]:
def make_carbon_footprints(key, building):
  building['footprint']['electric_co2'] = round((building['consumption_totals']['electric_consumption'] * carbon_cost['electricity']['carbon']), 2)
  building['footprint']['heating_co2'] = round((building['consumption_totals']['heat_consumption'] * carbon_cost['gas']['carbon']), 2)
  building['footprint']['water_co2'] = round((building['consumption_totals']['water_consumption'] * carbon_cost['water']['carbon']), 2)
  building['footprint']['total'] = round((building['footprint']['electric_co2'] + building['footprint']['heating_co2'] +  building['footprint']['water_co2']), 2)

  print(f"{key} - Carbon Footprint")
  print(f"Electric Carbon Footprint: {building['footprint']['electric_co2']}")
  print(f"Heating Carbon Footprint: {building['footprint']['heating_co2']}")
  print(f"Water Carbon Footprint: {building['footprint']['water_co2']}")
  print(f"Total Carbon Footprint: {building['footprint']['total']}")
  print("===================================================")

In [113]:
  for key, building in buildings.items():
    make_carbon_footprints(key, building)

PGB - Carbon Footprint
Electric Carbon Footprint: 136607.38
Heating Carbon Footprint: 96382.32
Water Carbon Footprint: 0.0
Total Carbon Footprint: 232989.7
DH - Carbon Footprint
Electric Carbon Footprint: 51436.62
Heating Carbon Footprint: 61611.2
Water Carbon Footprint: 179.04
Total Carbon Footprint: 113226.86
KH - Carbon Footprint
Electric Carbon Footprint: 24388.54
Heating Carbon Footprint: 3855.65
Water Carbon Footprint: 30.32
Total Carbon Footprint: 28274.51
FB - Carbon Footprint
Electric Carbon Footprint: 112368.54
Heating Carbon Footprint: 27245.94
Water Carbon Footprint: 37633.82
Total Carbon Footprint: 177248.3


In [114]:
def make_energy_cost_total(key, building):
  building['cost_totals']['electric_cost'] = round((building['consumption_totals']['electric_consumption'] * energy['electricity']['price']),2)
  building['cost_totals']['heating_cost'] = round((building['consumption_totals']['heat_consumption'] * energy['gas']['price']),2)
  building['cost_totals']['water_cost'] = round((building['consumption_totals']['water_consumption'] * energy['water']['price']),2)
  building['cost_totals']['total_cost'] = round(( building['cost_totals']['electric_cost'] +  building['cost_totals']['heating_cost'] + building['cost_totals']['water_cost'] ),2)
  print(f"{key} - Total Energy Finanical Costs")
  print(f"Electric Cost: £ {building['cost_totals']['electric_cost']}")
  print(f"Heating Cost: £ {building['cost_totals']['heating_cost']}")
  print(f"Water Cost: £ {building['cost_totals']['water_cost']}")
  print(f"Total Cost: £ {building['cost_totals']['total_cost']}")
  print("===================================================")

In [115]:
  for key, building in buildings.items():
    make_energy_cost_total(key, building)

PGB - Total Energy Finanical Costs
Electric Cost: £ 318514.63
Heating Cost: £ 57293.94
Water Cost: £ 0.0
Total Cost: £ 375808.57
DH - Total Energy Finanical Costs
Electric Cost: £ 119929.95
Heating Cost: £ 36624.43
Water Cost: £ 444.59
Total Cost: £ 156998.97
KH - Total Energy Finanical Costs
Electric Cost: £ 56864.48
Heating Cost: £ 2291.97
Water Cost: £ 75.29
Total Cost: £ 59231.74
FB - Total Energy Finanical Costs
Electric Cost: £ 261999.19
Heating Cost: £ 16196.2
Water Cost: £ 93453.12
Total Cost: £ 371648.51


In [116]:
## Remove datasets from buildings so can write to json
buildings['PGB']['dataset'] = ""
buildings['DH']['dataset'] = ""
buildings['KH']['dataset'] = ""
buildings['FB']['dataset'] = ""

In [117]:
file_path = '/content/drive/My Drive/buildings.json'

with open(file_path, 'w') as json_file:
    json.dump(buildings, json_file, indent=4)